In [1]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import re
import csv
import pandas as pd
import re
from dateutil import parser

import smtplib

In [2]:
# getting initial data pattern url, 10 pages loop

lista_links=[]
lista_nomes=[]
lista_nomes_links=[]


# i= numero de pagina a serem pesquisadas, ex: pagina 1 a 11
# esse for ira percorrer as paginas de noticias e encontrar as noticias que contem o termo pesquisado, ex: Reitoria
# em seguida ira entrar no link de cada uma e contar quantas vezes a palavra buscada aparece, ex: UFSM
# por fim, salva o titulo da noticia, link, data e hora; e o numero de apariçoes que a palavra buscada(UFSM) aparece

for i in range(1,11):
    URL = f'https://www.ufsm.br/category/noticias/page/{i}'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    
    articles = soup1.find_all('div', class_='col-lg-10 info-busca-lista')
    
    #print(articles)
    #print(len(articles))
    
    
    # getting values
    for article in articles:      
        # pega titulos dos artigos
        titles = article.find('a')
        #print(titles)
        
        # pegando href dos links
        # pegando nomes da noticias, do alt
        links = []
        alt_nomes=[]
        
        
        links.append(str(article.find('a')['href']))
        alt_nomes.append(str(article.find('a')['alt']))
        #print(links)
        #print(alt_nomes)
        
        
        # procura por termo relevante nas listas
        for i in range(0,len(links)):
            combined = '\t'.join(links)
            if('reitoria' in combined):
                #print('tem reitoria')
                
                lista_links.append(str(links))
                lista_nomes.append(str(alt_nomes))

                lista_nomes_links.append([lista_nomes, lista_links])
                
            else:
                pass
                #print('não tem reitoria')
                


In [3]:
# prints para verficação
        
print('\nTamanho da lista com termo pesquisado: \n',len(lista_nomes_links))
#print('\nLista com termo pesquisado: \n',lista_nomes_links)
print('\nLista de links: \n',lista_links)
print('\nLista de nomes: \n',lista_nomes)


# salvando as duas lista juntas, nome e link, em um data frame
#### A SER USADO BLOCOS ABAIXOS ####
#df = pd.DataFrame(lista_nomes_links, columns=['Nome Noticia', 'Link Noticia'])
#print('\n\nData Frame Link e nome noticia\n\n\n',df)


Tamanho da lista com termo pesquisado: 
 6

Lista de links: 
 ["['https://www.ufsm.br/2022/02/23/reitoria-estabelece-jornada-de-trabalho-dos-dias-25-02-e-02-03']", "['https://www.ufsm.br/2021/12/17/relatorio-de-gestao-2018-2021-reune-acoes-projetos-e-iniciativas-da-reitoria-da-ufsm']", "['https://www.ufsm.br/2021/06/24/pesquisa-de-opiniao-junto-a-comunidade-academica-para-sucessao-da-reitoria-acontece-hoje']", "['https://www.ufsm.br/2021/06/21/ufsm-promove-segunda-live-para-apresentacao-de-propostas-da-chapa-que-concorre-a-sucessao-da-reitoria']", "['https://www.ufsm.br/2021/05/21/ufsm-solicita-atualizacao-de-dados-no-portal-para-participar-da-pesquisa-de-opiniao-para-escolha-da-reitoria']", "['https://www.ufsm.br/2021/04/28/consu-deflagra-processo-eleitoral-para-sucessao-da-reitoria-da-ufsm']"]

Lista de nomes: 
 ["['Reitoria estabelece jornada de trabalho dos dias 25/02 e 02/03']", "['Relatório de Gestão 2018-2021 reúne ações, projetos e iniciativas da Reitoria da UFSM']", "['Pesqui

In [4]:

lista_dates=[]
lista_hora=[]
lista_dates_hora=[]
lista_count_palavra_procurada=[]


# contadar quantas vezes uma palvra aparece dentro do texto
palavra_procurada = 'UFSM'


# acessando cada pagina salva com o termo
for url in lista_links:
    URL_pagina_pesquisada = url
    URL_pagina_pesquisada = re.sub(r'\'', '', URL_pagina_pesquisada)
    URL_pagina_pesquisada = re.sub(r'\n', '', URL_pagina_pesquisada)
    URL_pagina_pesquisada = re.sub('\[', '', URL_pagina_pesquisada)
    URL_pagina_pesquisada = re.sub('\]', '', URL_pagina_pesquisada)
    #print(URL_pagina_pesquisada)
    
    # setando infos da pagina buscada anteriormente
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    pagina_pesquisada = requests.get(URL_pagina_pesquisada, headers=headers)

    soup_pagina_pesquisada1 = BeautifulSoup(pagina_pesquisada.content, "html.parser")

    soup_pagina_pesquisada2 = BeautifulSoup(soup_pagina_pesquisada1.prettify(), "html.parser")

    #print(soup_pagina_pesquisada2)
    
    
    texto_pagina_procurada = soup_pagina_pesquisada1.find_all('div', class_='entry-content noticia-conteudo')
    lista_count_palavra_procurada.append(str(texto_pagina_procurada).count(palavra_procurada))
    print(lista_count_palavra_procurada)
    
    
    
    # extraindo datas da pagina
    lista_dates=[]
    lista_hora=[]
    
    #date_news = soup_pagina_pesquisada1.find_all('span', class_='posted-on text-muted')
    date_news=soup_pagina_pesquisada1.find(class_='entry-date published updated')
    #print(date_news)
    
    # seleciona data se for post único OU se teve atualização; salva na lista de datas
    for t in (soup_pagina_pesquisada1.find_all('time', attrs={'class':'entry-date published updated'}) or soup_pagina_pesquisada1.find_all('time', attrs={'class':'entry-date published'})):
        date_time_str = t.get('datetime')
        date_time = parser.parse(date_time_str)
        
        lista_dates.append(lista_dates.append(date_time.date()))
        lista_dates.append(lista_dates.append(date_time.time()))
        #lista_hora.append(lista_hora.append(date_time.time()))
        
        lista_dates_hora.append(lista_dates)
        #lista_dates_hora.append(lista_hora)
        
        #print(lista_dates)
        #print(lista_hora)
        #print(date_time.date())
        #print(date_time.time())

 

[1]
[1, 18]
[1, 18, 3]
[1, 18, 3, 4]
[1, 18, 3, 4, 3]
[1, 18, 3, 4, 3, 4]


In [5]:
   

#------------------------------------
# print testes

print('\n\n------------------------------------\n\n')
print('\nTamanho da lista com termo pesquisado: \n',len(lista_nomes_links))
#print('\nLista com termo pesquisado: \n',lista_nomes_links)
print('\nLista de links: \n', lista_links)
print('\nLista de nomes: \n', lista_nomes)
print('\nLista de Dates: \n', lista_dates)
print('\nLista de Hora: \n', lista_hora)
print('\nLista de Date Hora: \n', lista_dates_hora)
print('\nContador de palavra pesquisada: \n', lista_count_palavra_procurada)
print('\n\n------------------------------------\n\n')

#------------------------------------




------------------------------------



Tamanho da lista com termo pesquisado: 
 6

Lista de links: 
 ["['https://www.ufsm.br/2022/02/23/reitoria-estabelece-jornada-de-trabalho-dos-dias-25-02-e-02-03']", "['https://www.ufsm.br/2021/12/17/relatorio-de-gestao-2018-2021-reune-acoes-projetos-e-iniciativas-da-reitoria-da-ufsm']", "['https://www.ufsm.br/2021/06/24/pesquisa-de-opiniao-junto-a-comunidade-academica-para-sucessao-da-reitoria-acontece-hoje']", "['https://www.ufsm.br/2021/06/21/ufsm-promove-segunda-live-para-apresentacao-de-propostas-da-chapa-que-concorre-a-sucessao-da-reitoria']", "['https://www.ufsm.br/2021/05/21/ufsm-solicita-atualizacao-de-dados-no-portal-para-participar-da-pesquisa-de-opiniao-para-escolha-da-reitoria']", "['https://www.ufsm.br/2021/04/28/consu-deflagra-processo-eleitoral-para-sucessao-da-reitoria-da-ufsm']"]

Lista de nomes: 
 ["['Reitoria estabelece jornada de trabalho dos dias 25/02 e 02/03']", "['Relatório de Gestão 2018-2021 reúne ações, projetos e inic

In [7]:


# criando lista geral
    
# salvando as listas juntas, nome , link e data em um data frame
df = pd.DataFrame(
    {
    'Nome Noticia':lista_nomes,
    'Link Noticia':lista_links,
    'Data e hora': lista_dates_hora,
    'Apariçoes da palavra pesquisada':lista_count_palavra_procurada
})
print('\n\nData lista_nomes_links_datas\n\n\n', df)  

# salvando DF em um arquivo csv

df.to_csv('Web scraping University news.csv')



Data lista_nomes_links_datas


                                         Nome Noticia  \
0  ['Reitoria estabelece jornada de trabalho dos ...   
1  ['Relatório de Gestão 2018-2021 reúne ações, p...   
2  ['Pesquisa de Opinião junto à Comunidade Acadê...   
3  ['UFSM promove segunda live para apresentação ...   
4  ['UFSM solicita atualização de dados no Portal...   
5  ['Consu deflagra processo eleitoral para suces...   

                                        Link Noticia  \
0  ['https://www.ufsm.br/2022/02/23/reitoria-esta...   
1  ['https://www.ufsm.br/2021/12/17/relatorio-de-...   
2  ['https://www.ufsm.br/2021/06/24/pesquisa-de-o...   
3  ['https://www.ufsm.br/2021/06/21/ufsm-promove-...   
4  ['https://www.ufsm.br/2021/05/21/ufsm-solicita...   
5  ['https://www.ufsm.br/2021/04/28/consu-deflagr...   

                          Data e hora  Apariçoes da palavra pesquisada  
0  [2022-02-23, None, 18:37:58, None]                                1  
1  [2021-12-17, None, 10:07:32, No